# Desarrollo y fundamentos del código implementado:


## Estimadores Generales de Densidad Espectral de Potencia (DEP)



### 1. Fundamentos: 

La **Densidad Espectral de Potencia (DEP)**, o *Power Spectral Density (PSD)*, es una función que describe cómo se distribuye la potencia media de una señal de tiempo discreto \( x[n] \) a través de las frecuencias \( f \).

El análisis se centra en aproximar la DEP teórica, la cual, para una señal que no varía, está ligada a su función de autocorrelación \( r_{xx}[k] \) por el **Teorema de Wiener-Khinchin**:

$$
P_{xx}(f) = \mathcal{F}\{r_{xx}[k]\}
$$

Al trabajar con un número finito de muestras (\( N \)), todo estimador se enfrenta a un **compromiso**:

| Métrica | Definición | Consecuencia |
|----------|-------------|---------------|
| **Resolución Espectral** | Capacidad de discernir componentes de frecuencia cercanas. Es inversamente proporcional a la longitud de la señal utilizada. | Aumenta con \( N \) (Periodograma). |
| **Varianza (Consistencia)** | Confiabilidad o estabilidad de la estimación. Mide la dispersión del estimador respecto al valor verdadero. | Disminuye con el promediado (Welch) o el suavizado (Blackman-Tukey). |

---

### 2. Estimador I: Periodograma Ventaneado

#### 2.1. Principio y Limitación (Inconsistencia)

El **Periodograma** es el estimador más básico y directo, calculado mediante la **Transformada Rápida de Fourier (FFT)** de la señal completa. Ofrece la máxima resolución espectral posible al utilizar toda la longitud \( N \) de la señal.

El Periodograma es un **estimador inconsistente**: su varianza no converge a cero al aumentar \( N \), lo que genera un espectro “ruidoso” o con fluctuaciones significativas que no representan la potencia real de la señal.

#### 2.2. Atenuación de la Fuga Espectral (Ventaneado)

Para señales finitas, la truncación de \( x[n] \) en el tiempo equivale a una **convolución con una función sinc** en la frecuencia, lo que introduce **fuga espectral (*spectral leakage*)**.

La implementación utiliza una **función de ventana** \( w[n] \) (por ejemplo, Hamming) en la función `periodogram_estimator` para suavizar los bordes de la señal y atenuar los lóbulos laterales del espectro, mejorando la calidad de la estimación y reduciendo la interferencia entre bandas.

#### 2.3. Fundamento Matemático

La estimación del **Periodograma ventaneado**, \( \hat{P}_{per}(f) \), se define como:

$$
\hat{P}_{per}(f) = \frac{1}{U} \left| \sum_{n=0}^{N-1} x[n] w[n] e^{-j2\pi fn/f_s} \right|^2
$$

donde el factor de normalización \( U \) es:

$$
U = \sum_{n=0}^{N-1} w^2[n]
$$

Este factor hace que la salida sea una verdadera **Densidad Espectral de Potencia** \((V^2/Hz)\).

---

### 3. Estimador II: Método de Welch (`welch_estimator`)

#### 3.1. Principio y Solución (Consistencia)

El **Método de Welch** es el estándar para la estimación de DEP, ya que resuelve el problema de inconsistencia del Periodograma mediante el **promediado**. Su implementación (`welch_estimator`) se basa en:

1. Dividir la señal \( x[n] \) en \( K \) segmentos más cortos de longitud \( L \) (*nperseg*).  
2. Aplicar una ventana \( w[n] \) a cada segmento.  
3. Calcular el Periodograma para cada segmento.  
4. Promediar los \( K \) Periodogramas.

#### 3.2. Justificación Matemática y Parámetros

El promediado estadístico de \( K \) periodogramas \( \hat{P}_{per,i}(f) \) reduce la varianza del estimador resultante en un factor de \( 1/K \), logrando una estimación **consistente y suave**:

$$
\hat{P}_{Welch}(f) = \frac{1}{K} \sum_{i=0}^{K-1} \hat{P}_{per,i}(f)
$$

**Parámetros clave:**

- **nperseg (Longitud L): Una \( L \) grande (segmentos largos) mejora la resolución, pero reduce \( K \) y el efecto de reducción de la varianza.  
- **noverlap (Solapamiento):** Típicamente 50%. Asegura que la ventana afecte uniformemente todos los puntos de la señal, mejorando la calidad del promedio sin redundancia significativa.

---

### 4. Estimador III: Blackman-Tukey (`blackman_tukey_estimator`)

#### 4.1. Principio y Fundamento

El método **Blackman-Tukey (BT)** es un estimador indirecto que se implementa en la función `blackman_tukey_estimator`, realizando la **Transformada de Fourier de la autocorrelación estimada** $( \hat{r}_{xx}[k] )$ ventaneada, según el **Teorema de Wiener-Khinchin**.
`Esta función la obtuve del libro de`**Monson Hayes - Statistical Digital Signal Processing and Modeling. En su página 421, figura 8.18**,`da un algoritmo hecho en Matlab para aplicar el método de Blackman-Tukey. En los códigos implementados para todas las señales recibidas en archivos, traduje ese código en Matlab a Python.`


Pasos:

1. **Eliminación de la componente DC:**  
   Se centra la señal:  
   $$ x'[n] = x[n] - \bar{x} $$

2. **Cálculo de la autocorrelación:**  
   $$ \hat{r}_{xx}[k] = \text{estimación de la autocorrelación de } x'[n] $$

3. **Ventaneado en el retardo:**  
   Aplicación de una ventana \( w_{auto}[k] \) a la autocorrelación truncada.

4. **Transformada de Fourier:**  
   $$ \hat{P}_{BT}(f) = \mathcal{F}\{\hat{r}_{xx}[k] \cdot w_{auto}[k]\} $$

#### 4.2. Suavizado y Parámetros

La principal ventaja de Blackman-Tukey es que el ventaneado aplicado a la autocorrelación produce un **espectro suavizado y estable** (baja varianza).  
El suavizado se logra mediante la **convolución del espectro con el espectro de la ventana** $( w_{auto}[k])$.

El parámetro **M (retardos)** define la longitud máxima de la autocorrelación utilizada y controla la resolución:

- Si \( M \to N \): alta resolución, pero alta varianza (similar al Periodograma).  
- Si \( M \) es pequeño: baja resolución, pero espectro muy suave (baja varianza).

---

### 5. Función Ancho de Banda (`bandwidth`)

La función `bandwidth` proporciona una métrica para cuantificar la **ocupación espectral** de la señal, independientemente del estimador (Periodograma, Welch, Blackman-Tukey) utilizado.

#### 5.2. Umbral Relativo

El **ancho de banda** se define mediante un umbral relativo \( \theta \) (típicamente 0.05 o 5%) para enfocarse solo en las frecuencias que portan potencia significativa, ignorando el ruido de fondo.

Primero se normaliza la DEP al máximo valor:

$$
P_{xx,\text{norm}}(f) = \frac{P_{xx}(f)}{\max(P_{xx}(f))}
$$

Luego, el ancho de banda (BW) se calcula como el rango donde la potencia normalizada supera el umbral \( \theta \):

$$
BW = f_{high} - f_{low} \quad \text{donde} \quad P_{xx,\text{norm}}(f) > \theta
$$

Esto permite una **comparación** del espectro entre diferentes señales (ECG, audio, RF, etc.) o entre diferentes estimadores.

---
